# Часть 1. Преобразование в эмбеддинги

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer

Оригинальный датасет:

In [35]:
df_raw = pd.read_csv("h_raw_data.1500k.records.csv")
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 22 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   record_id         1500000 non-null  object 
 1   time_created      1500000 non-null  object 
 2   time_updated      1500000 non-null  object 
 3   meta_data         0 non-null        float64
 4   page_number       1500000 non-null  int64  
 5   deactivated       583990 non-null   object 
 6   country_id        1500000 non-null  int64  
 7   country_title     200589 non-null   object 
 8   city_id           1500000 non-null  int64  
 9   city_title        236058 non-null   object 
 10  about             4778 non-null     object 
 11  activities        4288 non-null     object 
 12  books             3395 non-null     object 
 13  games             3463 non-null     object 
 14  interests         4656 non-null     object 
 15  education_form    22250 non-null    object 
 16  

Уберём дубликаты и возьмём только те записи, где есть "faculty_name":

In [36]:
df = df_raw.dropna(subset=["faculty_name"])
df.drop(
    columns=['record_id', 'time_created', 'time_updated', 'meta_data', 'page_number', 'deactivated'],
    inplace=True
)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30602 entries, 0 to 30601
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_id        30602 non-null  int64 
 1   country_title     27064 non-null  object
 2   city_id           30602 non-null  int64 
 3   city_title        25595 non-null  object
 4   about             442 non-null    object
 5   activities        576 non-null    object
 6   books             351 non-null    object
 7   games             330 non-null    object
 8   interests         563 non-null    object
 9   education_form    18595 non-null  object
 10  education_status  9453 non-null   object
 11  university_id     30602 non-null  int64 
 12  university_name   28694 non-null  object
 13  faculty_id        30602 non-null  int64 
 14  faculty_name      30602 non-null  object
 15  graduation_year   30602 non-null  int64 
dtypes: int64(5), object(11)
memory usage: 3.7+ MB


C:\Users\alice\AppData\Local\Temp\ipykernel_13764\2352650626.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(
C:\Users\alice\AppData\Local\Temp\ipykernel_13764\2352650626.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


Столбики, которые будем склеивать в единый текст:

In [37]:
features = ['country_title', 'city_title', 'faculty_name', 'about', 'activities', 'books', 'games', 'interests']

Чтобы их склеить без проблем, нужно запонить пропуски:

In [38]:
for col in features:
    df[col].fillna('', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30602 entries, 0 to 30601
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_id        30602 non-null  int64 
 1   country_title     30602 non-null  object
 2   city_id           30602 non-null  int64 
 3   city_title        30602 non-null  object
 4   about             30602 non-null  object
 5   activities        30602 non-null  object
 6   books             30602 non-null  object
 7   games             30602 non-null  object
 8   interests         30602 non-null  object
 9   education_form    18595 non-null  object
 10  education_status  9453 non-null   object
 11  university_id     30602 non-null  int64 
 12  university_name   28694 non-null  object
 13  faculty_id        30602 non-null  int64 
 14  faculty_name      30602 non-null  object
 15  graduation_year   30602 non-null  int64 
dtypes: int64(5), object(11)
memory usage: 3.7+ MB


C:\Users\alice\AppData\Local\Temp\ipykernel_13764\3121670749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].fillna('', inplace=True)


Собственно само склеивание в тексты:

In [39]:
texts = []
for i in range(len(df)):
    res = ''
    for col in features:
        if df.loc[i, col] == '':
            continue
        else:
            res += f"{df.loc[i, col]}, "
    texts.append(res)
texts

['Таиланд, Bangkok, คณะสัตวศาสตร์และเทคโนโลยีการเกษตร (Faculty of Animal Science and Agricultural Technology), ',
 'Россия, Усть-Кут, Факультет судовождения, ',
 'Россия, Симферополь, Экономический факультет, ',
 'Сальвадор, San Salvador, Facultad de Medicina, ',
 'Конструкторский факультет, ',
 'Россия, Воронеж, Экономический факультет, ',
 'Таиланд, Bangkok, Music, ',
 'Испания, Valencia, Facultat de Química, ',
 'Факультет архитектуры, строительства и дизайна, ',
 'Россия, Рубцовск, Кафедра общественных дисциплин, ',
 'Россия, Санкт-Петербург, Биологический факультет, ',
 'Россия, Красноярск, Центр дополнительного профессионального образования, ',
 'Россия, Москва, Факультет трубопроводного транспорта, ',
 'Россия, Москва, Геолого-географический факультет, ',
 'Кыргызстан, Джалал-Абад, Инженерно-педагогический факультет, ',
 'Турция, Güzel Sanatlar Fakültesi, ',
 'Россия, Грозный, Факультет физической культуры и спорта, Что хотите узнать?, Омон Россгвардия, Работа, ',
 'Пакистан, Ra

Пример результата:

In [40]:
texts[300]

'Казахстан, Алматы, Экономический факультет, '

Если нужно добавить такой столбик, то делаем:

df_raw["text"] = texts

Преобразуем такие тексты в эмбеддинги:

In [41]:
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2") # Сюда можно вставлять любой другой sentence transformer
embeddings = embedding_model.encode(texts, convert_to_tensor=True, show_progress_bar=True)
embeddings

Batches:   0%|          | 0/957 [00:00<?, ?it/s]

tensor([[ 0.1996,  0.2046,  0.0602,  ..., -0.2617,  0.1112,  0.1449],
        [-0.0385,  0.1080, -0.2511,  ..., -0.2245,  0.1558,  0.1864],
        [ 0.0635,  0.0491, -0.2541,  ..., -0.2343,  0.1690, -0.0175],
        ...,
        [-0.0103,  0.0170, -0.2174,  ..., -0.1568,  0.1184, -0.0067],
        [ 0.4426, -0.3400, -0.0660,  ...,  0.0600,  0.0936,  0.1715],
        [ 0.0528, -0.0696, -0.1860,  ..., -0.2139,  0.4114,  0.0262]],
       device='cuda:0')

Размер одного эмебеддинга:

In [42]:
len(embeddings[0])

384

Если нужно добавить такой столбик, то делаем:

df_raw["embedding"] = embeddings

# Инференс

In [43]:
from sentence_transformers import util
import torch

In [44]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [45]:
embeddings = embeddings.to(device)

In [46]:
query = "Россия, Красноярск, математика, ML, DOTA2"
query_embedding = embedding_model.encode(query, convert_to_tensor=True, show_progress_bar=True)
query_embedding = query_embedding.to(device)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
hits = util.semantic_search(query_embedding, embeddings, top_k=20)[0]
hits

[{'corpus_id': 2814, 'score': 0.8526118397712708},
 {'corpus_id': 26098, 'score': 0.8478858470916748},
 {'corpus_id': 16165, 'score': 0.8451181054115295},
 {'corpus_id': 13832, 'score': 0.8442667722702026},
 {'corpus_id': 18433, 'score': 0.82593834400177},
 {'corpus_id': 15197, 'score': 0.8183075785636902},
 {'corpus_id': 12861, 'score': 0.814052164554596},
 {'corpus_id': 18997, 'score': 0.8109022378921509},
 {'corpus_id': 27825, 'score': 0.8107451796531677},
 {'corpus_id': 17417, 'score': 0.8101609945297241},
 {'corpus_id': 29955, 'score': 0.8101609945297241},
 {'corpus_id': 169, 'score': 0.8083336353302002},
 {'corpus_id': 3229, 'score': 0.8071107268333435},
 {'corpus_id': 24136, 'score': 0.8071107268333435},
 {'corpus_id': 19489, 'score': 0.8069537878036499},
 {'corpus_id': 9073, 'score': 0.8062546849250793},
 {'corpus_id': 29378, 'score': 0.8062546253204346},
 {'corpus_id': 6963, 'score': 0.8047357797622681},
 {'corpus_id': 17662, 'score': 0.8047357797622681},
 {'corpus_id': 22276,

In [48]:
idx = [hit['corpus_id'] for hit in hits]
idx

[2814,
 26098,
 16165,
 13832,
 18433,
 15197,
 12861,
 18997,
 27825,
 17417,
 29955,
 169,
 3229,
 24136,
 19489,
 9073,
 29378,
 6963,
 17662,
 22276]

In [49]:
print("Вот что мы тебе рекомендуем:")
for n, id in enumerate(idx):
  print(f"""{n+1}. {df.loc[id, "university_name"]}, {df.loc[id, "faculty_name"]}""")

Вот что мы тебе рекомендуем:
1. Филиал МГОУ  им. В. С. Черномырдина, Прикладная математика
2. Институт (ф) МГОУ, Прикладная математика
3. Филиал ЧГУ, Математический
4. КрасГУ, Факультет математики и информатики
5. ВФ ННГУ им. Н. И. Лобачевского, Механико-математический факультет
6. НГПУ им. Козьмы Минина, Факультет математики, информатики и физики
7. БашГУ, Математический факультет
8. ХГУ им. Б. Гафурова (бывш. ЛГПИ им. Кирова), Факультет математики
9. СГУ им. Н. Г. Чернышевского, Механико-математический факультет
10. КГПУ им. В. П. Астафьева (бывш. КГПИ), Институт математики, физики и информатики
11. КГПУ им. В. П. Астафьева (бывш. КГПИ), Институт математики, физики и информатики
12. КемГУ, Математический факультет
13. ТвГУ, Математический факультет
14. ТвГУ, Математический факультет
15. ЯрГУ им. П. Г. Демидова, Математический факультет
16. ВГУ, Факультет прикладной математики, информатики и механики
17. ВГУ, Факультет прикладной математики, информатики и механики
18. АмГУ, Факультет 

In [50]:
for id in idx:
  print(df.loc[id])

country_id                                            1
country_title                                    Россия
city_id                                              68
city_title                                       Ковров
about                                                  
activities                                             
books                                                  
games                                                  
interests                                              
education_form                   Дистанционное обучение
education_status                                    NaN
university_id                                     18705
university_name     Филиал МГОУ  им. В. С. Черномырдина
faculty_id                                       300660
faculty_name                      Прикладная математика
graduation_year                                    2011
Name: 2814, dtype: object
country_id                              1
country_title                      Р